In [101]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import tiktoken
import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
import pandas as pd

In [198]:
df2 = pd.read_csv(r'E:\GRAD\validation.csv')
df2.to_excel('E:/GRAD/validation.xlsx')

In [206]:
# df = pd.read_csv(r'E:\GRAD\MODIF validation.csv')
df = pd.read_excel(r'E:\GRAD\new data.xlsx')
df

,Unnamed: 0,id,title,context,question,answers
0,0,621723207492,حمزة بن عبد المطلب,حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحا...,من هو حمزة بن عبد المطلب؟,{'text': array(['صحابي من صحابة رسول الإسلام م...
1,1,189105393656,حمزة بن عبد المطلب,حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحا...,بما وصفه رسول الله حمزه بن عبد المطلب؟,"{'text': array(['وَخَيْرُ أَعْمَامِي'], dtype=..."
2,2,662616978980,حمزة بن عبد المطلب,حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحا...,بما وصف رسول الله على ؟,"{'text': array(['«خَيْرُ إِخْوَتِي عَلِيٌّ،'],..."
3,3,50146585922,حمزة بن عبد المطلب,أسلم حمزة في السنة الثانية من بعثة النبي محمد،...,متى اسلم حمزة؟,{'text': array(['في السنة الثانية من بعثة النب...
4,4,259563807541,حمزة بن عبد المطلب,أسلم حمزة في السنة الثانية من بعثة النبي محمد،...,و ماذا فعل حمزه فى غزوة بدر؟,{'text': array(['وقَتَلَ فيها شيبة بن ربيعة مب...
...,...,...,...,...,...,...
697,697,470812622665,السودان,استقل السودان عن بريطانيا و مصر في الأول من ين...,متى استعلت الحرب الأهلية في السودان؟,"{'text': array(['منذ قبيل إعلان الاستقلال'], d..."
698,698,950438628756,السودان,استقل السودان عن بريطانيا و مصر في الأول من ين...,متى توقفت الحرب الأهلية في السودان؟,"{'text': array(['2005'], dtype=object), 'answe..."
699,699,296368076837,السودان,تكررت الإنقلابات العسكرية في تاريخ السودان الح...,في أي سنة وقع آخر انقلاب عسكري في السودان؟,"{'text': array(['عام 1989'], dtype=object), 'a..."
700,700,349611596646,السودان,تكررت الإنقلابات العسكرية في تاريخ السودان الح...,من قاد الانقلاب العسكري؟,"{'text': array(['العميد عمر البشير'], dtype=ob..."


In [207]:
contexts = df['context'].unique()
titles = df['title'].unique()

In [208]:
contexts = contexts.tolist()

In [209]:
title_list = []
for i in range(len(contexts)):
    for j in range(len(df['title'])):
        if contexts[i] == df['context'][j]:
            title_list.append(df['title'][j])
            break

In [210]:
len(title_list)

234

In [211]:
title_list[3]

'القمر'

In [182]:
contexts[3]

'القمر هو القمر الطبيعي الوَحيد للأرض  بالإضافة إلى أنه خامس أكبر قمرٍ طبيعيٍ في المجموعة الشمسية. فهو يُعَدُ أكبر قمرٍ طبيعيٍ في المجموعة الشمسية من ناحية نسبة حجمه إلى كوكبه التابع له، حيث أن قطره يصل إلى ربع قطر الأرض، كما أن كتلته تصل إلى 1 على 81 من كتلة الأرض، هذا بالإضافة إلى أنه يُعَدُ ثاني أعلى قمرٍ من ناحية الكثافة بعد قمر إيو.'

In [212]:
from langchain.docstore.document import Document

document_list = []
for i in range(len(contexts)):
    context = contexts[i]
    title = title_list[i]
    document_list.append(Document(page_content=context, metadata={"title": title}))

In [213]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000,
        chunk_overlap=500,
        length_function=len,
        add_start_index=True,
    )
chunks = text_splitter.split_documents(document_list)
print(f"Split {len(document_list)} documents into {len(chunks)} chunks.")

Split 234 documents into 234 chunks.


In [106]:
!pip install langchain_mistralai

In [214]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [231]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_mistralai import ChatMistralAI

model_name2="intfloat/multilingual-e5-base"
model = model_name2.split('/')
# Get embeddings.
embedding_model = HuggingFaceEmbeddings(model_name=model_name2)

knowledge_base = FAISS.from_documents(chunks, embedding_model)

retriever = knowledge_base.as_retriever(search_type="mmr",
                    search_kwargs={'k': 5, 'lambda_mult': 0.5})
# template = """
# You are given a context this reteived from a database based on similarity between it and between the given question . Your role is to regenrate the answer of the qestion based on the provided context and please note to answer in arabic only:
template = """
You are given a context retrieved from a database based on the similarity between it and the given question. Your role is to generate the answer to the question based on the provided context. Please note to answer only in Arabic.

This is the retrieved context:
{context}


and here's the Question: {question}
"""


c:\Users\Osama\anaconda3\envs\GPU\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [232]:
prompt = ChatPromptTemplate.from_template(template)

#model = ChatOpenAI(model="", openai_api_key='sk-proj-SyR3MBymitFuPptyAiavT3BlbkFJ9DK477Yqu2HQaZFgEOjW')
import getpass
import os

os.environ["MISTRAL_API_KEY"] = getpass.getpass()
# rDptqgcc9pt8gepc0nKYzTKSKIsIhYc9

In [233]:
from operator import itemgetter
llm = ChatMistralAI(model="mistral-large-latest")
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [219]:
query = ' بما وصف رسول الله حمزه بن عبد المطلب؟'
print(chain.invoke(query))

وصف رسول الله صلّى الله عليه وسلّم حمزة بن عبد المطلب بأنه خير أعمامه لقوله: «خَيْرُ إِخْوَتِي عَلِيٌّ، وَخَيْرُ أَعْمَامِي حَمْزَةُ رَضِيَ اللَّهُ عَنْهُمَا».


In [190]:
query = "من هو حمزة بن عبد المطلب؟"
print(chain.invoke(query))

حمزة بن عبد المطلب هو صحابي من صحابة رسول الإسلام محمد، وعمُّه وأخوه من الرضاعة وأحد وزرائه الأربعة عشر. تربى حمزة في كنف والده عبد المطلب بن هاشم الذي كان سيد قريش وبني هاشم، ونشأ مع تِربه وابن أخيه عبد الله وأخيه من الرضاعة محمد بن عبد الله بن عبد المطلب. وعاشا ينهلان من الشمائل والقيم العربية الأصيلة، من بطولة وشجاعة وكرم ونجدة وغيرها، وارتبطت بينهما صداقة متينة ووثيقة العُرا. يُعتبر خير أعمام رسول الله صلى الله عليه وسلم لقوله: «خَيْرُ إِخْوَتِي عَلِيٌّ، وَخَيْرُ أَعْمَامِي حَمْزَةُ رَضِيَ اللَّهُ عَنْهُمَا».


In [235]:
from datasets import Dataset 
import os
#from ragas import evaluate
#from ragas.metrics import faithfulness, answer_correctness, answer_similarity, context_entity_recall, context_recall, context_precision, answer_relevancy
#from ragas.metrics.critique import harmfulness

questions = df['question'][:20].tolist()
from datasets import Dataset
answers=[]
for qus in questions:
    query=str(qus)
    response = chain.invoke(query)  # Using invoke() method for the chain
    answers.append(response)


HTTPStatusError: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}

In [ ]:
df

,id,title,context,question,answers
0,621723207492,حمزة بن عبد المطلب,حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحا...,من هو حمزة بن عبد المطلب؟,{'text': array(['صحابي من صحابة رسول الإسلام م...
1,189105393656,حمزة بن عبد المطلب,حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحا...,بما وصفه رسول الله؟,"{'text': array(['وَخَيْرُ أَعْمَامِي'], dtype=..."
2,662616978980,حمزة بن عبد المطلب,حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحا...,بما وصف رسول الله على ؟,"{'text': array(['«خَيْرُ إِخْوَتِي عَلِيٌّ،'],..."
3,50146585922,حمزة بن عبد المطلب,أسلم حمزة في السنة الثانية من بعثة النبي محمد،...,متى اسلم حمزة؟,{'text': array(['في السنة الثانية من بعثة النب...
4,259563807541,حمزة بن عبد المطلب,أسلم حمزة في السنة الثانية من بعثة النبي محمد،...,و ماذا فعل فى غزوة بدر؟,{'text': array(['وقَتَلَ فيها شيبة بن ربيعة مب...
...,...,...,...,...,...
697,470812622665,السودان,استقل السودان عن بريطانيا و مصر في الأول من ين...,متى استعلت الحرب الأهلية في السودان؟,"{'text': array(['منذ قبيل إعلان الاستقلال'], d..."
698,950438628756,السودان,استقل السودان عن بريطانيا و مصر في الأول من ين...,متى توقفت الحرب الأهلية في السودان؟,"{'text': array(['2005'], dtype=object), 'answe..."
699,296368076837,السودان,تكررت الإنقلابات العسكرية في تاريخ السودان الح...,في أي سنة وقع آخر انقلاب عسكري في السودان؟,"{'text': array(['عام 1989'], dtype=object), 'a..."
700,349611596646,السودان,تكررت الإنقلابات العسكرية في تاريخ السودان الح...,من قاد الانقلاب العسكري؟,"{'text': array(['العميد عمر البشير'], dtype=ob..."


In [ ]:
print(df['question'][1])
print(df['context'][1])
print(df['answers'][1])

بما وصفه رسول الله؟
حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحابة رسول الإسلام محمد، وعمُّه وأخوه من الرضاعة وأحد وزرائه الأربعة عشر، وهو خير أعمامه لقوله: «خَيْرُ إِخْوَتِي عَلِيٌّ، وَخَيْرُ أَعْمَامِي حَمْزَةُ رَضِيَ اللَّهُ عَنْهُمَا».
{'text': array(['وَخَيْرُ أَعْمَامِي'], dtype=object), 'answer_start': array([166], dtype=int32)}


In [221]:
answers

['حمزة بن عبد المطلب هو صحابي من صحابة رسول الإسلام محمد، وعمُّه وأخوه من الرضاعة وأحد وزرائه الأربعة عشر. تربى في كنف والده عبد المطلب بن هاشم الذي كان سيد قريش وبني هاشم. نشأ مع تِربه وابن أخيه عبد الله وأخيه من الرضاعة محمد بن عبد الله بن عبد المطلب، وعاشا ينهلان من الشمائل والقيم العربية الأصيلة، من بطولة وشجاعة وكرم ونجدة وغيرها. وارتبطت بينهما صداقة متينة ووثيقة العُرا. وهو خير أعمام النبي محمد لقوله: «خَيْرُ إِخْوَتِي عَلِيٌّ، وَخَيْرُ أَعْمَامِي حَمْزَةُ رَضِيَ اللَّهُ عَنْهُمَا».']

In [39]:
df["context"][i]

'حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحابة رسول الإسلام محمد، وعمُّه وأخوه من الرضاعة وأحد وزرائه الأربعة عشر، وهو خير أعمامه لقوله: «خَيْرُ إِخْوَتِي عَلِيٌّ، وَخَيْرُ أَعْمَامِي حَمْزَةُ رَضِيَ اللَّهُ عَنْهُمَا».'

In [79]:
df

,id,title,context,question,answers
0,621723207492,حمزة بن عبد المطلب,حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحا...,من هو حمزة بن عبد المطلب؟,{'text': array(['صحابي من صحابة رسول الإسلام م...
1,189105393656,حمزة بن عبد المطلب,حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحا...,بما وصفه رسول الله؟,"{'text': array(['وَخَيْرُ أَعْمَامِي'], dtype=..."
2,662616978980,حمزة بن عبد المطلب,حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحا...,بما وصف رسول الله على ؟,"{'text': array(['«خَيْرُ إِخْوَتِي عَلِيٌّ،'],..."
3,50146585922,حمزة بن عبد المطلب,أسلم حمزة في السنة الثانية من بعثة النبي محمد،...,متى اسلم حمزة؟,{'text': array(['في السنة الثانية من بعثة النب...
4,259563807541,حمزة بن عبد المطلب,أسلم حمزة في السنة الثانية من بعثة النبي محمد،...,و ماذا فعل فى غزوة بدر؟,{'text': array(['وقَتَلَ فيها شيبة بن ربيعة مب...
...,...,...,...,...,...
697,470812622665,السودان,استقل السودان عن بريطانيا و مصر في الأول من ين...,متى استعلت الحرب الأهلية في السودان؟,"{'text': array(['منذ قبيل إعلان الاستقلال'], d..."
698,950438628756,السودان,استقل السودان عن بريطانيا و مصر في الأول من ين...,متى توقفت الحرب الأهلية في السودان؟,"{'text': array(['2005'], dtype=object), 'answe..."
699,296368076837,السودان,تكررت الإنقلابات العسكرية في تاريخ السودان الح...,في أي سنة وقع آخر انقلاب عسكري في السودان؟,"{'text': array(['عام 1989'], dtype=object), 'a..."
700,349611596646,السودان,تكررت الإنقلابات العسكرية في تاريخ السودان الح...,من قاد الانقلاب العسكري؟,"{'text': array(['العميد عمر البشير'], dtype=ob..."


In [225]:
# Assuming `answers` contains the predicted answers and `df` contains the ground truth answers
num_samples = min(len(answers), len(df["answers"]))
exact_matches = []
f1_scores = []

for i in range(num_samples):
    pred_answer = answers[i]
    gt_answer = df["answers"][i]
    
    # Calculate exact match
    exact_match = 1 if pred_answer.strip().lower() == gt_answer.strip().lower() else 0
    exact_matches.append(exact_match)
    
    # Calculate F1 score
    pred_tokens = pred_answer.strip().lower().split()
    gt_tokens = gt_answer.strip().lower().split()
    common_tokens = set(pred_tokens) & set(gt_tokens)
    precision = len(common_tokens) / len(pred_tokens) if len(pred_tokens) > 0 else 0
    recall = len(common_tokens) / len(gt_tokens) if len(gt_tokens) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    f1_scores.append(f1)

    # Print prediction and metrics
    print(f"Prediction: {pred_answer}")
    print(f"Exact Match Score: {exact_match:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print()

# Calculate average exact match and F1 score
avg_exact_match = sum(exact_matches) / num_samples
avg_f1_score = sum(f1_scores) / num_samples

print(f"Average Exact Match Score: {avg_exact_match:.4f}")
print(f"Average F1 Score: {avg_f1_score:.4f}")


Prediction: حمزة بن عبد المطلب هو صحابي من صحابة رسول الإسلام محمد، وعمُّه وأخوه من الرضاعة وأحد وزرائه الأربعة عشر. تربى في كنف والده عبد المطلب بن هاشم الذي كان سيد قريش وبني هاشم، ونشأ مع تِربه وابن أخيه عبد الله وأخيه من الرضاعة محمد بن عبد الله بن عبد المطلب، وعاشا ينهلان من الشمائل والقيم العربية الأصيلة، من بطولة وشجاعة وكرم ونجدة وغيرها، وارتبطت بينهما صداقة متينة ووثيقة العُرا. وهو خير أعمام رسول الإسلام محمد لقوله: «خَيْرُ إِخْوَتِي عَلِيٌّ، وَخَيْرُ أَعْمَامِي حَمْزَةُ رَضِيَ اللَّهُ عَنْهُمَا».
Exact Match Score: 0.0000
F1 Score: 0.2115

Prediction: وصف رسول الله حمزة بن عبد المطلب بأنه "خير أعمامي" حسب ما يروى في الحديث: «خَيْرُ إِخْوَتِي عَلِيٌّ، وَخَيْرُ أَعْمَامِي حَمْزَةُ رَضِيَ اللَّهُ عَنْهُمَا».
Exact Match Score: 0.0000
F1 Score: 0.0000

Prediction: وصف رسول الله عليه السلام في السيرة النبوية أنه كان نائمًا في الحجر فيزعجه جبريل عليه السلام بهمزته بقدمه، وكان رسول الله يرى دابة بيضاء بين البغل والحمار وفي فخذيها جناحان تحفز بهما رجليه. ولم يذكر النص أي وصف خاص لرسو

In [71]:
answers

['حمزة بن عبد المطلب هو صحابي من صحابة رسول الإسلام محمد، وعمُّه وأخوه من الرضاعة وأحد وزرائه الأربعة عشر، وهو خير أعمامه لقوله: «خَيْرُ إِخْوَتِي عَلِيٌّ، وَخَيْرُ أَعْمَامِي حَمْزَةُ رَضِيَ اللَّهُ عَنْهُمَا». تربى حمزة بن عبد المطلب في كنف والده عبد المطلب بن هاشم الذي كان سيد قريش وبني هاشم، ونشأ مع تِربه وابن أخيه عبد الله وأخيه من الرضاعة محمد بن عبد الله بن عبد المطلب، وعاشا ينهلان من الشمائل والقيم العربية الأصيلة، من بطولة وشجاعة وكرم ونجدة وغيرها، وارتبطت بينهما صداقة متينة ووثيقة العُرا.',
 'وصفه رسول الله بأنها المانعة المنجية تنجيه من عذاب القبر.',
 'رسول الله صلّى الله عليه وسلّم وُصف بأنه نائم في الحجر، ثم أتاه جبريل عليه السلام وهمزه بقدمه، فجلس رسولنا الكريم فلم يرَ شيئا، ثم عاد إلى مضجعه، فأتاه مرّةً ثانية فهمزه بقدمه، فجلس ولم يرَ شيئا، ثم عاد مرّةً أخرى إلى مضجعه، فأتاه مرّةً ثالثة فهمزه بقدمه، فجلس رسولنا الكريم، وأخذ جبريل بعضده، وحينها قام رسول الله معه، وخرج به جبريل إلى باب المسجد، فإذا رسول الله يرى دابّةً بيضاء بين البغل والحمار، في فخذيها جناحان تحفّز بهما ر

In [50]:
for i in range(0, len(questions)): 
    print(i)       
    data = {
        "question": [questions[i]],
        "answer":[answers[i]] ,
        "contexts": [[str(df["context"][i])]],
        "ground_truth":[df["answers"][i]]
    }

    dataset = Dataset.from_dict(data)
    score = evaluate(dataset,metrics=[faithfulness, harmfulness, answer_relevancy, answer_correctness, answer_similarity, context_precision, context_recall, context_entity_recall], raise_exceptions=False)
    print(score)

0


ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

In [ ]:
!pip install streamlit